# 🌐 Telco Network Performance Analysis

This notebook demonstrates how to analyze network performance data using Snowflake and AI capabilities for telecommunications operations.

## Objectives:
- Analyze network latency and throughput across regions
- Identify performance bottlenecks and optimization opportunities
- Use Cortex AI for predictive analytics on network performance
- Generate insights for network planning and investment decisions

In [ ]:
# Import required libraries
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("📊 Libraries imported successfully!")
print(f"🔗 Snowpark version: {snowpark.__version__}")

In [ ]:
# Get Snowpark session
session = snowpark.context.get_active_session()
print(f"❄️ Connected to Snowflake: {session.get_current_account()}")
print(f"🗄️ Current database: {session.get_current_database()}")
print(f"📂 Current schema: {session.get_current_schema()}")

## 📈 Network Performance Overview

In [ ]:
# Get network performance summary
print("📊 Analyzing network performance metrics...")

# Get current database and use correct schema for tables
current_db = session.get_current_database()
current_schema = session.get_current_schema()
# Tables are created in DEFAULT_SCHEMA, not the notebook schema (NOTEBOOKS)
data_schema = "DEFAULT_SCHEMA"

print(f"🔍 Notebook running in schema: {current_schema}")
print(f"📊 Querying data from schema: {data_schema}")

network_summary_query = f"""
SELECT 
    network_type,
    region,
    COUNT(DISTINCT cell_tower_id) as tower_count,
    AVG(latency_ms) as avg_latency_ms,
    AVG(throughput_mbps) as avg_throughput_mbps,
    AVG(packet_loss_percent) as avg_packet_loss,
    AVG(uptime_percent) as avg_uptime
FROM {current_db}.{data_schema}.NETWORK_PERFORMANCE 
WHERE measurement_timestamp >= DATEADD(month, -3, CURRENT_DATE())
GROUP BY network_type, region
ORDER BY network_type, avg_latency_ms
"""

try:
    network_summary_df = session.sql(network_summary_query).to_pandas()
    print("✅ Network Performance Summary:")
    display(network_summary_df)
    
    if network_summary_df.empty:
        print("📭 No network performance data found. Please check if data has been loaded.")
        print("💡 Run the configure_attendee_account.template.sql script to load sample data.")
        
except Exception as e:
    print(f"❌ Error executing query: {e}")
    print(f"🔍 Trying to query: {current_db}.{data_schema}.NETWORK_PERFORMANCE")
    print("💡 Solutions:")
    print("1. Run configure_attendee_account.template.sql to create tables and load data")
    print("2. Verify you have SELECT permissions on the NETWORK_PERFORMANCE table")
    print(f"3. Check that tables exist in {data_schema} schema (not {current_schema})")
    print("4. Ensure you're connected to the correct database")

In [ ]:
# Create network performance dashboard
print("📈 Creating network performance dashboard...")

try:
    if 'network_summary_df' in locals() and not network_summary_df.empty:
        # Create simple bar chart for latency by region
        fig = px.bar(network_summary_df, 
                    x='REGION', 
                    y='AVG_LATENCY_MS',
                    color='NETWORK_TYPE',
                    title='🌐 Average Network Latency by Region',
                    labels={'AVG_LATENCY_MS': 'Average Latency (ms)', 'REGION': 'Region'})
        
        fig.update_layout(height=500, width=900)
        fig.show()
        
        print("\n💡 Network Performance Insights:")
        print("📊 Analyze latency patterns across regions")
        print("🚀 Identify high-throughput network types")
        print("⚠️ Monitor packet loss for quality issues")
        print("⏰ Track uptime for SLA compliance")
        
    else:
        print("⚠️ No network performance data available for visualization")
        print("🔧 Please ensure the NETWORK_PERFORMANCE table exists and contains data")
        print("📋 Sample data structure expected:")
        print("   - NETWORK_TYPE (text)")
        print("   - REGION (text)")
        print("   - CELL_TOWER_ID (integer)")
        print("   - LATENCY_MS (decimal)")
        print("   - THROUGHPUT_MBPS (decimal)")
        print("   - PACKET_LOSS_PERCENT (decimal)")
        print("   - UPTIME_PERCENT (decimal)")
        print("   - MEASUREMENT_TIMESTAMP (timestamp)")
        
except Exception as e:
    print(f"❌ Error creating visualization: {e}")
    print("💡 Make sure the previous cell executed successfully")